In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json
import torch

In [2]:
import dataset
import algorithm

In [3]:
CONFIG = json.load(open('config.json'))

In [4]:
data = dataset.download_dataset()

Into folder: Dataset/Not/Argetina
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND1.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
		Images div: 215173/537932, 40%
		Images div: 268966/537932, 50%
		Images div: 322760/537932, 60%
		Images div: 376553/537932, 70%
		Images div: 430346/537932, 80%
		Images div: 484139/537932, 90%
		Images div: 537932/537932, 100%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND2.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
		Images div: 215173/537932, 40%
		Images div: 268966/537932, 50%
		Images div: 322760/537932, 60%
		Images div: 376553/537932, 70%
		Images div: 430346/537932, 80%
		Images div: 484139/537932, 90%
		Images div: 537932/537932, 100%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND3.tif
		Fil

In [5]:
print(len(data))

1054793


In [6]:
normalized_data = dataset.Normalizer.transform(data)

In [7]:
data_x, data_y = dataset.split_data_x_y(normalized_data)

In [8]:
print(data_y)

[1 1 1 ... 0 1 1]


In [9]:
data_x_train, data_x_test = dataset.split_data_train_test(data_x, CONFIG["data"]["train_split_size"])
data_y_train, data_y_test = dataset.split_data_train_test(data_y, CONFIG["data"]["train_split_size"])

In [10]:
data_x_train.shape

(843834, 20, 20, 4, 3)

In [11]:
dataset_train = dataset.TimeSeriesDataset(data_x_train, data_y_train)
dataset_test = dataset.TimeSeriesDataset(data_x_test, data_y_test)

In [12]:
train_dataloader = DataLoader(dataset_train, CONFIG["training"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(dataset_test, CONFIG["training"]["batch_size"], shuffle=True)

In [13]:
convnet = algorithm.CNNModel(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(convnet.parameters(), lr=0.001, momentum=0.9)

In [15]:
# Entrenamiento del modelo
for epoch in range(CONFIG["training"]["num_epoch"]):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(CONFIG["training"]["device"])
        labels = labels.to(CONFIG["training"]["device"])
        # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = convnet(inputs)
        loss = criterion(outputs, labels.type(torch.LongTensor).cuda())
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.693
[1,  4000] loss: 0.691
[1,  6000] loss: 0.684
[1,  8000] loss: 0.637
[1, 10000] loss: 0.567
[1, 12000] loss: 0.544
[1, 14000] loss: 0.523
[1, 16000] loss: 0.519
[1, 18000] loss: 0.508
[1, 20000] loss: 0.505
[1, 22000] loss: 0.501
[1, 24000] loss: 0.505
[1, 26000] loss: 0.499
[1, 28000] loss: 0.497
[1, 30000] loss: 0.487
[1, 32000] loss: 0.506
[1, 34000] loss: 0.489
[1, 36000] loss: 0.497
[1, 38000] loss: 0.493
[1, 40000] loss: 0.490
[1, 42000] loss: 0.497
[1, 44000] loss: 0.487
[1, 46000] loss: 0.489
[1, 48000] loss: 0.484
[1, 50000] loss: 0.478
[1, 52000] loss: 0.478
[1, 54000] loss: 0.483
[1, 56000] loss: 0.486
[1, 58000] loss: 0.478
[1, 60000] loss: 0.473
[1, 62000] loss: 0.485
[1, 64000] loss: 0.476
[1, 66000] loss: 0.481
[1, 68000] loss: 0.473
[1, 70000] loss: 0.480
[1, 72000] loss: 0.480
[1, 74000] loss: 0.472
[1, 76000] loss: 0.466
[1, 78000] loss: 0.463
[1, 80000] loss: 0.466
[1, 82000] loss: 0.474
[1, 84000] loss: 0.458
[1, 86000] loss: 0.454
[1, 88000] 

[4, 70000] loss: 0.385
[4, 72000] loss: 0.383
[4, 74000] loss: 0.395
[4, 76000] loss: 0.381
[4, 78000] loss: 0.381
[4, 80000] loss: 0.387
[4, 82000] loss: 0.386
[4, 84000] loss: 0.378
[4, 86000] loss: 0.391
[4, 88000] loss: 0.382
[4, 90000] loss: 0.389
[4, 92000] loss: 0.385
[4, 94000] loss: 0.387
[4, 96000] loss: 0.382
[4, 98000] loss: 0.384
[4, 100000] loss: 0.375
[4, 102000] loss: 0.392
[4, 104000] loss: 0.386
[4, 106000] loss: 0.377
[4, 108000] loss: 0.383
[4, 110000] loss: 0.381
[4, 112000] loss: 0.396
[4, 114000] loss: 0.390
[4, 116000] loss: 0.382
[4, 118000] loss: 0.379
[4, 120000] loss: 0.385
[4, 122000] loss: 0.382
[4, 124000] loss: 0.392
[4, 126000] loss: 0.383
[4, 128000] loss: 0.377
[4, 130000] loss: 0.375
[4, 132000] loss: 0.388
[4, 134000] loss: 0.394
[4, 136000] loss: 0.382
[4, 138000] loss: 0.381
[4, 140000] loss: 0.383
[4, 142000] loss: 0.394
[4, 144000] loss: 0.390
[4, 146000] loss: 0.376
[4, 148000] loss: 0.374
[4, 150000] loss: 0.374
[4, 152000] loss: 0.379
[4, 154

In [16]:
correct = 0
total = 0
with torch.no_grad():
    for idx, (images, labels) in enumerate(test_dataloader):
        images = images.to(CONFIG["training"]["device"])
        labels = labels.to(CONFIG["training"]["device"])
        outputs = convnet(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {0} test images: {1} %'.format(total, 100*correct/total))

Accuracy of the network on the 210959 test images: 82.02920946724245 %


In [18]:
torch.save(convnet.state_dict(), './ConvNet.pth')

In [17]:
for idx, (x, y) in enumerate(test_dataloader):
    x = x.to(CONFIG["training"]["device"])
    y = y.to(CONFIG["training"]["device"])
    out_batch = convnet(x)
    _, predicted = torch.max(out_batch, 1)
    predicted = predicted.cpu().numpy()
    label = y.cpu().numpy()
    for i in range(len(label)):
        print('Label predict: {0}, Label real: {1}'.format(predicted[i], label[i]))
    if idx == 20:
        break

Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 1, Label real: 0
Label predict: 1, Label real: 1
Label predict: 1, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 1, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 1, Label real: 1
Label predict: 1, Label real: 1
Label predict: 0, Label real: 0
Label predict: 1, Label real: 1
Label predict: 1, Label real: 1
Label predict: 1, Label real: 1
Label predict: 0, Label real: 0
Label predict: 1, Label real: 1
Label predict: 1, Label real: 1
Label predict: 0, Label real: 0
Label predict: 1, Label real: 1
Label predict: 1, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 1, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 1, Label real: 1
Label pr